# Deep Learning Regularization Techniques

Regularisation techniques are essential in deep learning to prevent overfitting, ensuring that models generalize well to new, unseen data. Overfitting occurs when a model learns the training data too well, capturing noise in the training data as if it were a true pattern. This notebook explores two popular regularisation techniques: Dropout and Batch Normalisation.

This notebook explores two common regularization techniques used in Deep Learning: Dropout and Batch Normalization. These techniques help address the problem of overfitting, which can significantly impact the performance of deep neural networks.

<img src="./imgs/overfit_vs_underfit.webp" alt="drawing" width="725"/>

## 1. Dropout

Dropout is a straightforward yet effective regularization technique. By randomly "dropping out" a proportion of neurons in the network during training, it prevents the network from becoming too dependent on any single neuron. This randomness encourages the network to develop more robust features that are not reliant on specific paths, enhancing generalization to new data.

**Concept:**

* During training, a random subset of neurons in a layer is temporarily ignored (dropped out) with a predefined probability (e.g., 0.5).
* This forces the remaining neurons to learn independently and become more robust to the absence of their neighbors.
* At test time, all neurons are included, but their activations are scaled by the dropout rate (e.g., multiplied by 0.5) to account for the neurons that were dropped during training.

**Benefits:**

* Reduces overfitting by preventing co-adaptation of features.
* Improves generalization performance on unseen data.
* Encourages robustness by making the network less reliant on specific neurons.

![dropout](imgs/dropout.gif)


In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Sample data (adjust for your actual dataset)
x_train = tf.random.normal((1000, 20)) 
y_train = tf.random.uniform((1000,), maxval=10, dtype=tf.int32) 

# Model with Dropout
model = Sequential([
    Dense(64, activation='relu', input_shape=(20,)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, validation_split=0.2)


Epoch 1/10
25/25 [==============================] - 0s 4ms/step - loss: 2.5463 - accuracy: 0.1287 - val_loss: 2.3784 - val_accuracy: 0.0950
Epoch 2/10
25/25 [==============================] - 0s 1ms/step - loss: 2.4082 - accuracy: 0.1412 - val_loss: 2.3579 - val_accuracy: 0.1050
Epoch 3/10
25/25 [==============================] - 0s 1ms/step - loss: 2.4279 - accuracy: 0.1125 - val_loss: 2.3489 - val_accuracy: 0.0900
Epoch 4/10
25/25 [==============================] - 0s 1ms/step - loss: 2.4053 - accuracy: 0.1063 - val_loss: 2.3361 - val_accuracy: 0.0900
Epoch 5/10
25/25 [==============================] - 0s 4ms/step - loss: 2.3515 - accuracy: 0.1213 - val_loss: 2.3318 - val_accuracy: 0.1000
Epoch 6/10
25/25 [==============================] - 0s 1ms/step - loss: 2.3500 - accuracy: 0.1287 - val_loss: 2.3274 - val_accuracy: 0.1000
Epoch 7/10
25/25 [==============================] - 0s 1ms/step - loss: 2.3399 - accuracy: 0.1262 - val_loss: 2.3282 - val_accuracy: 0.0900
Epoch 8/10
25/25 [==

## 2. Batch Normalization

Batch Normalisation is another powerful technique that normalizes the inputs of each layer to have a mean of 0 and a standard deviation of 1. This normalization helps to stabilize and accelerate the training process, combating issues related to poor initialization and helping gradients flow more smoothly through the network.

**Concept:**

* During training, for each mini-batch, Batch Normalization subtracts the mean and divides by the standard deviation of the activations of each layer.
* This normalizes the activations to a zero mean and unit variance.
* The layer then applies learned scale and shift factors to recover the original activation distribution if desired.

**Benefits:**

* Stabilizes the training process by making the activations less sensitive to initialization and weight updates.
* Improves gradient flow, allowing for faster training and potentially higher accuracy.
* Reduces the need for heavy weight initialization schemes.

![batch_norm](imgs/batch_norm.webp)


In [2]:
from tensorflow.keras.layers import BatchNormalization

# Model with Batch Normalization
model = Sequential([
    Dense(64, activation='relu', input_shape=(20,)),
    BatchNormalization(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, validation_split=0.2)


Epoch 1/10
25/25 [==============================] - 0s 4ms/step - loss: 2.8193 - accuracy: 0.1125 - val_loss: 2.3756 - val_accuracy: 0.1200
Epoch 2/10
25/25 [==============================] - 0s 1ms/step - loss: 2.5058 - accuracy: 0.1363 - val_loss: 2.3756 - val_accuracy: 0.1050
Epoch 3/10
25/25 [==============================] - 0s 1ms/step - loss: 2.3121 - accuracy: 0.1775 - val_loss: 2.3914 - val_accuracy: 0.1000
Epoch 4/10
25/25 [==============================] - 0s 1ms/step - loss: 2.2082 - accuracy: 0.2050 - val_loss: 2.4111 - val_accuracy: 0.1100
Epoch 5/10
25/25 [==============================] - 0s 1ms/step - loss: 2.1033 - accuracy: 0.2450 - val_loss: 2.4324 - val_accuracy: 0.1150
Epoch 6/10
25/25 [==============================] - 0s 1ms/step - loss: 2.0668 - accuracy: 0.2512 - val_loss: 2.4600 - val_accuracy: 0.1100
Epoch 7/10
25/25 [==============================] - 0s 1ms/step - loss: 2.0149 - accuracy: 0.2887 - val_loss: 2.4870 - val_accuracy: 0.1100
Epoch 8/10
25/25 [==

## How to Choose Between Dropout and Batch Normalization

Choosing the right regularization technique is crucial for the success of your deep learning model. While Dropout and Batch Normalization can both improve model generalization, they do so in different ways and have unique considerations. This section will guide you through choosing the most appropriate regularization technique for your specific scenario.


### Considerations for Dropout

Dropout randomly deactivates a subset of neurons in the network during training, which helps prevent overfitting by ensuring that no single neuron can overly influence the output. It is particularly effective in large networks where overfitting is a significant concern. However, Dropout might not be as beneficial in models that are already small or in cases where every neuron is crucial for the task.


#### When to Use Dropout

- In deep neural networks prone to overfitting.
- In layers with a large number of neurons.
- As a complementary technique to other forms of regularization.

### Considerations for Batch Normalization

Batch Normalization standardizes the inputs to a layer for each mini-batch, stabilizing the learning process and reducing the number of epochs required to train deep networks. It is especially useful when training deep networks with complex architectures. Unlike Dropout, Batch Normalization can sometimes lead to improved performance even in smaller networks.

#### When to Use Batch Normalization

- To improve training stability and speed.
- In very deep networks where vanishing or exploding gradients are a concern.
- Before activation functions, to normalize inputs.

### Combining Dropout and Batch Normalization

In practice, Dropout and Batch Normalization can be combined to leverage the strengths of both techniques. However, the layer order and configuration play a crucial role in how effective the combination is. A common approach is to apply Batch Normalization before activation functions and Dropout after activation functions or in specific layers where overfitting is more likely.


In [3]:
# Example of combining Batch Normalization and Dropout in a model layer
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

### Practical Tips for Regularization

Implementing regularization techniques effectively requires understanding not just when but also how to use them. Here are some practical tips:

- Start with a small amount of Dropout (e.g., 0.2 to 0.5) and adjust based on validation performance.
- Use Batch Normalization liberally in deep networks to stabilize training, but be mindful of its impact on inference time.
- Experiment with combining both techniques, monitoring model performance and training stability.
- Remember, regularization is just one part of model development. Model architecture, data preprocessing, and training procedure also play critical roles in building a robust model.